In [1]:
import irep
import pandas as pd

In [2]:
df = pd.read_csv('../datasets/house-votes-84.csv')

In [9]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.33, random_state=42)

In [3]:
irep_cls = irep.IREP(class_feat='Party',pos_class='democrat')
irep_cls

<IREP object not fit>

In [10]:
irep_cls.fit(train, seed=42, display=False)

In [11]:
irep_cls.ruleset

<Ruleset object: [[physician-fee-freeze=n^adoption-of-the-budget-resolution=y]v[physician-fee-freeze=n^synfuels-corporation-cutback=y]v[education-spending=n^el-salvador-aid=n^duty-free-exports=y]v[synfuels-corporation-cutback=y^immigration=n^adoption-of-the-budget-resolution=y]v[physician-fee-freeze=n^education-spending=y]v[education-spending=n^immigration=n^Water-project-cost-sharing=n]v[synfuels-corporation-cutback=y^adoption-of-the-budget-resolution=?]v[export-administration-act-south-africa=?^mx-missile=y]v[synfuels-corporation-cutback=y^export-administration-act-south-africa=?^Handicapped-infants=y]v[synfuels-corporation-cutback=y^el-salvador-aid=n^adoption-of-the-budget-resolution=n]v[duty-free-exports=y^Water-project-cost-sharing=y^anti-satellite-test-ban=n]v[adoption-of-the-budget-resolution=y^immigration=n^anti-satellite-test-ban=n]v[synfuels-corporation-cutback=y^export-administration-act-south-africa=n^Handicapped-infants=n]]>

In [113]:
len(irep_cls.ruleset.rules)

13

In [14]:
from sklearn.metrics import precision_score, recall_score
X_test = test.drop('Party',axis=1)
y_test = test['Party']

irep_precision = irep_cls.score(X_test, y_test, precision_score)
irep_recall = irep_cls.score(X_test, y_test, recall_score)
print(f'IRE: precision {irep_precision}, recall {irep_recall}')

IRE: precision 0.9313725490196079, recall 0.979381443298969


In [84]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()

In [90]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
df = pd.read_csv('../datasets/house-votes-84.csv')
le = LabelEncoder()
df_le=df.apply(le.fit_transform)
enc = OneHotEncoder(sparse=False)
enc.fit(df_onehot)
df_hot=enc.transform(df_le)
df_hot.shape

(435, 50)

In [95]:
train, test = train_test_split(df_hot, test_size=.33)#, random_state=42)
train_X = train[:,2:]
train_y = train[:,0]
test_X = test[:,2:]
test_y = test[:,0]
print(train_X.shape, train_y.shape)
print(test_X.shape, test_y.shape)

(291, 48) (291,)
(144, 48) (144,)


In [96]:
tree.fit(train_X, train_y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [97]:
predictions = tree.predict(test_X)
predictions

array([1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1.,
       0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1.,
       1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0.,
       0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1.,
       0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0.,
       1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1.,
       1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0.,
       1., 1., 1., 1., 1., 0., 1., 1.])

In [98]:
tree_precision = precision_score(test_y, predictions)
tree_recall = recall_score(test_y, predictions)
print(f'tree: precision {tree_precision}, recall {tree_recall}')

tree: precision 0.9310344827586207, recall 0.9101123595505618


In [110]:
tree.tree_.node_count

35

In [112]:
tree.tree_.node_count**(1/2)

5.916079783099616

In [157]:
def give_reasons(irep_, df):
    def pos_reasons(example):
        print(example)
        assert len(example)==1
        return [rule for rule in irep_.ruleset.rules if len(rule.covers(example))==1]
    
    return [pos_reasons(df[df.index==i]) for i in df.index]

In [158]:
give_reasons(irep_cls, df)

[[],
 [],
 [<Rule object: [synfuels-corporation-cutback=y^immigration=n^adoption-of-the-budget-resolution=y]>,
  <Rule object: [adoption-of-the-budget-resolution=y^immigration=n^anti-satellite-test-ban=n]>],
 [<Rule object: [physician-fee-freeze=n^adoption-of-the-budget-resolution=y]>,
  <Rule object: [physician-fee-freeze=n^synfuels-corporation-cutback=y]>,
  <Rule object: [synfuels-corporation-cutback=y^immigration=n^adoption-of-the-budget-resolution=y]>,
  <Rule object: [adoption-of-the-budget-resolution=y^immigration=n^anti-satellite-test-ban=n]>],
 [<Rule object: [physician-fee-freeze=n^adoption-of-the-budget-resolution=y]>,
  <Rule object: [physician-fee-freeze=n^synfuels-corporation-cutback=y]>,
  <Rule object: [synfuels-corporation-cutback=y^immigration=n^adoption-of-the-budget-resolution=y]>,
  <Rule object: [duty-free-exports=y^Water-project-cost-sharing=y^anti-satellite-test-ban=n]>,
  <Rule object: [adoption-of-the-budget-resolution=y^immigration=n^anti-satellite-test-ban=n